# 🎓 Travaux Pratiques : Introduction Pédagogique aux GANs (MLP-GAN)

**Auteur :**Benlahmar Habib


## Objectifs Pédagogiques de l'Atelier

1.  **Conceptualisation :** Comprendre le principe du **jeu antagoniste** et l'analogie du Faux-Monnayeur vs. Détective.
2.  **Théorie :** Décortiquer la fonction de coût **Minimax** qui régit l'apprentissage des GANs.
3.  **Implémentation Basique :** Mettre en œuvre un GAN simple (basé sur des couches linéaires/MLP) avec PyTorch sur la base Fashion-MNIST. 
4.  **Analyse :** Comprendre les défis liés à la stabilité de l'entraînement des GANs.

--- 

## I. Introduction Conceptuelle : Le Jeu Antagoniste

Les **Generative Adversarial Networks** (Réseaux Génératifs Antagonistes) sont composés de deux réseaux qui s'affrontent dans un jeu à somme nulle : le **Générateur (G)** et le **Discriminateur (D)**. 

| Composant | Rôle Analogue | Rôle Technique | Objectif | 
| :--- | :--- | :--- | :--- | 
| **Générateur (G)** | Le Faux-Monnayeur | Prend un bruit aléatoire $\mathbf{z}$ et crée $\mathbf{x}_{\text{faux}}$. | Tromper $D$ pour que $D$ classe les faux comme VRAIS (1). | 
| **Discriminateur (D)** | Le Détective | Classifie une image comme VRAIE (1) ou FAUSSE (0). | Identifier parfaitement la provenance de l'image (maximiser l'exactitude). | 

### 📌 1.1. La Fonction Objectif (Minimax Game)

L'entraînement du GAN est formulé comme la recherche d'un **équilibre de Nash** dans le jeu Minimax :

$$\min_G \max_D V(D, G) = \mathbb{E}_{\mathbf{x} \sim p_{\text{data}}(\mathbf{x})} [\log D(\mathbf{x})] + \mathbb{E}_{\mathbf{z} \sim p_{\mathbf{z}}(\mathbf{z})} [\log (1 - D(G(\mathbf{z})))]$$

* **$\max_D$ (Maximisation par D) :** Le Discriminateur veut que $D(\mathbf{x})$ (vrai) soit proche de 1 et que $D(G(\mathbf{z}))$ (faux) soit proche de 0. Cela maximise l'expression.
* **$\min_G$ (Minimisation par G) :** Le Générateur veut que $D(G(\mathbf{z}))$ (faux) soit proche de 1 (pour tromper $D$). Cela minimise le second terme de l'expression.

**L'équilibre idéal** est atteint lorsque $D(\mathbf{x}) = D(G(\mathbf{z})) = 0.5$ pour tout $\mathbf{x}$, car $D$ ne peut plus faire la différence.

### Question  (Q1.1)

Si, au début de l'entraînement, le Discriminateur renvoie $D(\mathbf{x}) = 0.9$ pour les vraies images et $D(G(\mathbf{z})) = 0.1$ pour les fausses images, comment cela impacte-t-il l'apprentissage du Générateur $G$ à ce stade ? (Indice : regardez le terme $\log (1 - D(G(\mathbf{z})))$).

--- 

## II. Préparation et Architecture (MLP-GAN)

Nous utilisons des réseaux de neurones multi-couches (MLP) simples pour démarrer. Les images 28x28 seront mises à plat en vecteurs de 784 dimensions.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from tqdm.notebook import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Exécution sur {device}")

# Hyperparamètres
image_size = 28 * 28  # 784
latent_dim = 100       # Dimension du bruit z
batch_size = 128
epochs = 50
lr = 0.0002 # Taux d'apprentissage souvent bas pour la stabilité des GANs

# Chargement des données et aplatissement des images (ToTensor() + normalisation [0, 1])
transform = ToTensor()
train_dataset = FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### 2.2. Le Générateur (G)

Input : Vecteur de bruit $z$ (`latent_dim`). Output : Vecteur image $\mathbf{x}'$ (`image_size`).

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, img_dim),
            nn.Sigmoid() # Sigmoid pour garantir la sortie [0, 1] (normalisation simple)
        )

    def forward(self, z):
        return self.model(z)

### 2.3. Le Discriminateur (D)

Input : Vecteur image $x$ (`image_size`). Output : Score scalaire (logit) qui sera transformé en probabilité [0, 1].

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(img_dim, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3), 
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1) # Sortie finale sans activation (logits)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Initialisation
G = Generator(latent_dim, image_size).to(device)
D = Discriminator(image_size).to(device)

# Optimiseurs : Le choix d'Adam et des betas=(0.5, 0.999) est standard pour les GANs.
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss : BCEWithLogitsLoss est recommandé pour la stabilité numérique. Elle inclut Sigmoid.
criterion = nn.BCEWithLogitsLoss()

# Bruit fixe pour suivre la progression de la génération
fixed_noise = torch.randn(64, latent_dim, device=device) 

# Fonction utilitaire de visualisation (réutilisée du TP VAE)
def show_grid(grid, title="", figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.title(title)
    # Note: On utilise 'cmap="gray"' car Fashion-MNIST est en niveaux de gris
    plt.imshow(np.transpose(grid.numpy(), (1, 2, 0)), cmap="gray")
    plt.axis("off")
    plt.show()

### Question  (Q2.1)

Observez les taux d'apprentissage. Pourquoi le taux d'apprentissage est-il souvent très bas (ici 0.0002) dans les GANs par rapport à d'autres réseaux (souvent 0.001) ? Quel risque prend-on avec des taux d'apprentissage trop élevés pour $G$ ou $D$ ?

--- 

## III. Boucle d'Entraînement : L'Alternance Cruciale

L'entraînement s'effectue en deux sous-étapes par batch : d'abord on améliore $D$, puis on améliore $G$. C'est le cœur du jeu antagoniste.

In [ ]:
def train_gan(G, D, G_optimizer, D_optimizer, criterion, dataloader, epochs, latent_dim, device):
    G.train(); D.train()
    for epoch in trange(epochs, desc="Entraînement GAN"):
        D_loss_total = 0
        G_loss_total = 0
        
        for real_images, _ in dataloader:
            
            # --- Préparation des données --- 
            real_images = real_images.view(-1, image_size).to(device)
            current_batch_size = real_images.size(0)
            
            # Tenseurs de cibles (VRAI=1.0, FAUX=0.0)
            # Utiliser des floats pour les cibles de BCE
            real_labels = torch.ones(current_batch_size, 1, device=device)
            fake_labels = torch.zeros(current_batch_size, 1, device=device)

            # ===============================================
            # 1. ÉTAPE D : Améliorer le Discriminateur
            # ===============================================
            D_optimizer.zero_grad()

            # 1a. Loss sur les images réelles (cible = 1)
            D_real_pred = D(real_images)
            D_real_loss = criterion(D_real_pred, real_labels)
            
            # 1b. Générer les fausses images et calculer la loss (cible = 0)
            noise = torch.randn(current_batch_size, latent_dim, device=device)
            fake_images = G(noise) 
            
            # ⚠️ POINT CLÉ : .detach() pour arrêter la rétropropagation vers G
            D_fake_pred = D(fake_images.detach()) 
            D_fake_loss = criterion(D_fake_pred, fake_labels)
            
            # 1c. Rétropropagation de la loss totale de D
            D_loss = D_real_loss + D_fake_loss
            D_loss.backward()
            D_optimizer.step()
            
            D_loss_total += D_loss.item()
            
            # ===============================================
            # 2. ÉTAPE G : Améliorer le Générateur
            # ===============================================
            G_optimizer.zero_grad()
            
            # G veut que D prédise 1 (VRAI) pour ses fausses images (pour le tromper)
            # On réutilise fake_images, mais cette fois le graphe de G est attaché.
            G_pred = D(fake_images) 
            G_loss = criterion(G_pred, real_labels) # ⚠️ CIBLE = 1 (real_labels) pour la loss de G
            
            G_loss.backward()
            G_optimizer.step()
            
            G_loss_total += G_loss.item()

        # --- Affichage de l'évolution (par époque) ---
        avg_D_loss = D_loss_total / len(dataloader)
        avg_G_loss = G_loss_total / len(dataloader)
        tqdm.write(f"Epoch {epoch+1:2d} | D Loss: {avg_D_loss:.4f} | G Loss: {avg_G_loss:.4f}")
        
        # 4. Visualisation toutes les 5 époques
        if (epoch + 1) % 5 == 0:
            G.eval()
            with torch.no_grad():
                # Générer 64 images à partir du bruit fixe
                generated_images = G(fixed_noise).cpu().view(64, 1, 28, 28)
                show_grid(make_grid(generated_images, 8), title=f"Génération Époque {epoch+1}")
            G.train()

# train_gan(G, D, G_optimizer, D_optimizer, criterion, train_dataloader, epochs, latent_dim, device) # <-- DÉCOMMENTER POUR LANCER L'ENTRAINEMENT

### Question  (Q3.1)

Dans l'étape d'entraînement du Discriminateur, nous utilisons `.detach()` sur les `fake_images` générées par $G$. Quel est l'effet précis de cette méthode PyTorch dans le contexte des GANs, et pourquoi est-il essentiel pour l'entraînement de $D$ ?

--- 

## IV. Analyse des Résultats et Questions de Synthèse

Une fois l'entraînement terminé, analysez les images générées et les courbes de perte.

### Questions Finales (Synthèse et Ouverture)

1.  **Interprétation de la Perte $D$ :** Expliquez la valeur idéale vers laquelle la perte du Discriminateur devrait converger. Que signifie une $D$ Loss trop basse (proche de 0) en milieu ou fin d'entraînement ?
2.  **Problème de Stabilité :** Le *Mode Collapse* est un défi majeur des GANs. Expliquez ce phénomène et décrivez visuellement ce qu'il se passerait sur la grille de génération si votre modèle en était victime.
3.  **Avantage vs. VAE :** Par rapport au VAE que vous avez étudié précédemment, quel est le principal avantage du GAN en termes de qualité visuelle des images générées ? Quelle en est la contrepartie (désavantage) ?
4.  **Ouverture DCGAN :** Quel type d'architecture sera introduit dans la prochaine étape (DCGAN) pour remplacer les couches linéaires (MLP) ? Quel avantage les couches convolutionnelles apportent-elles pour la génération d'images, par rapport aux couches linéaires ?